In [1]:
from elasticsearch import Elasticsearch
from dotenv import load_dotenv
import os 

load_dotenv()

True

In [2]:
os.chdir("D:\\elasticsearch_chatbot")

In [3]:
from elasticsearch import Elasticsearch, helpers

client = Elasticsearch(
    "https://608291f321b44714a3d68daf7c09ca10.us-central1.gcp.cloud.es.io:443",
    api_key=os.getenv("ELASTIC_SEARCH_API_KEY")
)

In [4]:
if client.ping():
    print("Connected to the ElasticSearch")
else:
    print("You haven't connected to ElasticSearch")

Connected to the ElasticSearch


In [5]:
import json 

index_name = "search-simbolo"

with open("Simbolo_Information.json", "rt") as file:
    simbolo_file = json.load(file)
    #print(simbolo_file)
    
    bulk_response = helpers.bulk(client, simbolo_file, index=index_name)
    print(bulk_response)

(34, [])


In [6]:
user_input = "Who is the founder of Simbolo"

query = {
    "query": {
        "match": {
            "Questions": user_input
        },
    }
}

client.count(index=index_name, body=query)

ObjectApiResponse({'count': 62, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [7]:
response = client.search(index=index_name, body=query, size=35)
results = response["hits"]["hits"]
# Print the documents
template_for_prompt = []
for result in results: 
    print(result["_source"])
    template_for_prompt.append(result['_source'])
#print(len(results))

{'Questions': 'How did the Founder Phyo Thu Htet die?', 'Answers': 'Because of the earthquake that was happened in March 29.'}
{'Questions': 'How did the Founder Phyo Thu Htet die?', 'Answers': 'Because of the earthquake that was happened in March 29.'}
{'Questions': 'How did the Founder Phyo Thu Htet die?', 'Answers': 'Because of the earthquake that was happened in March 29.'}
{'Questions': 'What is the location of Simbolo?', 'Answers': 'Simbolo is located at 91H*42H, Mahar Aung Myay, Mandalay. Let me know if you need directions!'}
{'Questions': 'What is the location of Simbolo?', 'Answers': 'Simbolo is located at 91H*42H, Mahar Aung Myay, Mandalay. Let me know if you need directions!'}
{'Questions': 'What is the location of Simbolo?', 'Answers': 'Simbolo is located at 91H*42H, Mahar Aung Myay, Mandalay. Let me know if you need directions!'}
{'Questions': 'What is Simbolo?', 'Answers': 'Símbolo is a training school which provides high-quality education in science and technology by off

In [8]:
prompt_template = [{
    "question": [item['Questions'] for item in template_for_prompt],
    "answer": [item['Answers'] for item in template_for_prompt]
}]

prompt_template_for_llm = """
You are a helpful assistant trained to answer questions about Simbolo, a training school.

You have access to the following knowledge base of Questions and Answers:

{question}
{answer}

When a user asks a question, provide the most accurate answer based on the knowledge base above.

If the question cannot be answered from the knowledge base, reply: "I'm sorry, I do not have enough information to answer that question."
"""

questions_text = "\n".join(prompt_template[0]['question'])
answers_text = "\n".join(prompt_template[0]['answer'])

final_prompt = prompt_template_for_llm.format(
    question=questions_text,
    answer=answers_text
)

final_prompt

'\nYou are a helpful assistant trained to answer questions about Simbolo, a training school.\n\nYou have access to the following knowledge base of Questions and Answers:\n\nHow did the Founder Phyo Thu Htet die?\nHow did the Founder Phyo Thu Htet die?\nHow did the Founder Phyo Thu Htet die?\nWhat is the location of Simbolo?\nWhat is the location of Simbolo?\nWhat is the location of Simbolo?\nWhat is Simbolo?\nWhat is Simbolo?\nWhat is Simbolo?\nWhat are the benefits that only students of Simbolo can enjoy?\nWhat are the benefits that only students of Simbolo can enjoy?\nWhat are the benefits that only students of Simbolo can enjoy?\nWhere can we contact the Simbolo?\nWhere can we contact the Simbolo?\nWhere can we contact the Simbolo?\nWhat is the course outline of Python?\nWhat is the course outline of UXUI?\nWhat is the course outline of Python?\nWhat is the course outline of UXUI?\nWhat is the course outline of Python?\nWhat is the course outline of UXUI?\nWhat is the course outline

In [9]:
from groq import Groq

model = Groq(
    api_key=os.getenv("GROQ_API_KEY"),
)

chat_completion = model.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": final_prompt
        },
        {
            "role": "user",
            "content": user_input,
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

The founder of Simbolo is Phyo Thu Htet.
